## Imports and configs

In [112]:
from dotenv import load_dotenv
import os
from pathlib import Path
from IPython.display import Markdown
from bs4 import BeautifulSoup
from tqdm import tqdm
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
load_dotenv()
MISTRAL_KEY = os.getenv("MISTRAL_KEY")

Useful:
* https://python.langchain.com/docs/concepts/retrieval/
* https://cameronrwolfe.substack.com/p/the-basics-of-ai-powered-vector-search?utm_source=profile&utm_medium=reader2
* https://python.langchain.com/docs/how_to/lcel_cheatsheet/
* https://python.langchain.com/docs/tutorials/rag/


Ideas:
1. Query re-writing (or just generate 5 different queries from user query - MultiQueryRetriever)
2. Use query step backor HyDE
3. Hybrid search (BM-25 + bi-encoders)
4. Retrieve not top-1, but 3-5 best matched documents and then process it
5. Splitting task is not obvious (depends on the model context size)!

## Data processing and indexing

Use such pipeline:
1. Load html-docs 
2. Preprocess it, using cusom func (transform html -> raw txt)
3. Use RecursiveSplitter
4. (*) Add the tags to each data chunk

### Loading data

In [39]:
data_folder = Path("/home/deniskirbaba/Documents/legal-ai/small_legal_html_dataset")
n_docs = len(list(data_folder.iterdir()))
f"Number of docs: {n_docs}"

'Number of docs: 9'

In [108]:
def parse_html(html: str) -> str:
    """
    Parse HTML doc to raw text. Also extract only data from <body> tags.

    Adds a new line for 'br',  'p', 'h1', 'h2', 'h3', 'h4','tr', 'th' tags
    and also a new line in front of text for li elements (bullet lists).
    """
    soup = BeautifulSoup(html, features="html.parser").body

    text = ''
    for e in soup.descendants:
        if isinstance(e, str):
            text += e.strip()
        elif e.name in ['br',  'p', 'h1', 'h2', 'h3', 'h4','tr', 'th']:
            text += '\n'
        elif e.name == 'li':
            text += '\n- '
    return text

In [109]:
processed_docs = []
for doc_path in tqdm(data_folder.iterdir(), total=n_docs):
    with open(doc_path, 'r') as f:
        html_doc = f.read()
    processed_docs.append(parse_html(html_doc))

100%|██████████| 9/9 [00:00<00:00, 38.95it/s]


In [111]:
Markdown(processed_docs[0][:2000])


Решение № 2-696/2023 от 23 августа 2023 г. по делу № 2-444/2023~М-344/2023Ярцевский городской суд (Смоленская область) - Гражданское
/
Дело № 2-696/2023

67RS0008-01-2023-000447-69


Р Е Ш Е Н И Е

Именем Российской Федерации

23 августа 2023 года г.Ярцево Смоленской области

Ярцевский городской суд Смоленской области:

в составе:

председательствующего судьи Семеновой Е.А.,

при секретаре Слесаревой М.В.,

рассмотрев в открытом судебном заседании гражданское дело по иску Общества с ограниченной ответственностью «Смоленская региональная теплоэнергетическая компания «Смоленскрегионтеплоэнерго»» к ФИО1, действующей в своих интересах и интересах несовершеннолетнего ФИО4, <дата> года рождения, ФИО2 о взыскании задолженности и пени за тепловую энергию,

УСТАНОВИЛ:

Общество с ограниченной ответственностью «Смоленская региональная теплоэнергетическая компания «Смоленскрегионтеплоэнерго»» (далее - ООО «Смоленскрегионтеплоэнерго») обратилось в суд, уточнив требования, с иском к ФИО1 и ФИО2 о взыскании задолженности и пени за тепловую энергию. В обоснование указало, истец является единой теплоснабжающей организацией на территории Смоленской области, осуществляющей поставку тепловой энергии, в том числе в многоквартирный дом, расположенный по адресу: Смоленская область, г.Ярцево, <адрес>, в целях оказания коммунальной услуги отопления собственникам и пользователям помещений в доме. Квартира №25, расположенная в вышеуказанном доме, площадью 29 кв.м, расчетной площадью 39,4 кв.м, находится в общей долевой собственности ФИО1, ФИО2 и несовершеннолетнего ФИО4, законным представителем которого является ФИО1 По степени благоустройства квартира обеспечена отоплением и горячим водоснабжением. В квартире установлены индивидуальные приборы учета ГВС. Обязательства по подаче тепловой энергии истцом были исполнены в полном объеме, ответчиками допущено нарушение обязательств по оплате потребленной тепловой энергии, в связи с чем, по состоянию на 04.04.2023 образовалась задолженность за фа

In [115]:
# create simple metadata list (later need to fill this with parsed tags from `sudact.ru`)
metadatas = [{"index": i, "length": len(doc)} for i, doc in enumerate(processed_docs)]

### Split docs

* https://python.langchain.com/docs/concepts/text_splitters/
* https://python.langchain.com/docs/how_to/recursive_text_splitter/
* https://python.langchain.com/docs/how_to/semantic-chunker/ - Interesting

In [128]:
rec_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, #  need to pick value based on context of our model
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False
)

In [129]:
chunks = rec_splitter.create_documents(processed_docs, metadatas)
f"{len(chunks)} chunks was construct from {len(processed_docs)} HTML docs"

'276 chunks was construct from 9 HTML docs'

In [ ]:
# let's observe the 1 HTML splits
for chunk in chunks:
    if chunk.metadata["index"] == 0:
        print("="*250)
        print(chunk.page_content)
    else:
        break

Решение № 2-696/2023 от 23 августа 2023 г. по делу № 2-444/2023~М-344/2023Ярцевский городской суд (Смоленская область) - Гражданское
/
Дело № 2-696/2023

67RS0008-01-2023-000447-69


Р Е Ш Е Н И Е

Именем Российской Федерации

23 августа 2023 года г.Ярцево Смоленской области

Ярцевский городской суд Смоленской области:

в составе:

председательствующего судьи Семеновой Е.А.,

при секретаре Слесаревой М.В.,

рассмотрев в открытом судебном заседании гражданское дело по иску Общества с ограниченной ответственностью «Смоленская региональная теплоэнергетическая компания «Смоленскрегионтеплоэнерго»» к ФИО1, действующей в своих интересах и интересах несовершеннолетнего ФИО4, <дата> года рождения, ФИО2 о взыскании задолженности и пени за тепловую энергию,

УСТАНОВИЛ:
Общество с ограниченной ответственностью «Смоленская региональная теплоэнергетическая компания «Смоленскрегионтеплоэнерго»» (далее - ООО «Смоленскрегионтеплоэнерго») обратилось в суд, уточнив требования, с иском к ФИО1 и ФИО2 о вз

### Embedding model

Which model to use? 
* russian-based or multilingual?

https://huggingface.co/spaces/mteb/leaderboard

Benchmarks: ruMTEB, encodechka.  
From lectures: **Качество моделей может сильно отличаться от домена к домену. Крайне рекомендуется дообучать энкодеры под свои домены.**

Can we finetune encoder? Do we have compute/data for this?

In [ ]:
# or we just gonna use openaiembeddings and we fine?

# from langchain_openai import OpenAIEmbeddings
# embeddings_model = OpenAIEmbeddings()
# embeddings = embeddings_model.embed_documents(chunks)
# query_embedding = embeddings_model.embed_query(user_query)

### Vector storage

Chroma, FAISS?  
As a understand, some of them are not only have indexing and vector storage, but all needed stuff for information retrieval (candidate-generation, ranking), so we can just use it

## Retrieval and generation

* Which LLM we are using?
* prompts
* user query magic (re-writing, multiple queries...)
* maybe predict some tags from query to improve performance (f.e. legal field: правовое, уголовное. как-то выделить категорию запроса или какие-то другие общие теги чтобы облегчить поиск по БД. потому что у нас же есть теги из данных с судебных практик, можем прям прогонять модель, которая будет делать мультитегирование запроса и потом можем даже фильтр в БД по этому сделать, т.к. теги будут в метаданных для каждого документа)
* (*) some advanced rags: self-rag, corrective-rag
